In [21]:
import os
import sys
sys.path.append('../code')
import importlib

import numpy as np  
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

import SeaLevelContrib as slc

In [26]:
importlib.reload(slc)

<module 'SeaLevelContrib' from '../code/SeaLevelContrib.py'>

In [12]:
tg_id=[20, 22, 23, 24, 25, 32]
gia_df = pd.DataFrame()
for idx, tgi in enumerate(tg_id):
    if idx == 0:
        gia_df = slc.GIA_ICE6G(tgi)
        gia_df = gia_df.rename(columns = {'GIA' : tgi})
    else:
        gia_df[tgi] = slc.GIA_ICE6G(tgi)
gia_df['mean'] = gia_df.mean(axis=1)
gia_df

,20,22,23,24,25,32,mean
time,,,,,,,
1900,0.000,0.000,0.00,0.000,0.000,0.000,0.000000
1901,0.004,0.017,0.05,0.061,0.058,0.031,0.036833
1902,0.008,0.034,0.10,0.122,0.116,0.062,0.073667
1903,0.012,0.051,0.15,0.183,0.174,0.093,0.110500
1904,0.016,0.068,0.20,0.244,0.232,0.124,0.147333
...,...,...,...,...,...,...,...
2015,0.460,1.955,5.75,7.015,6.670,3.565,4.235833
2016,0.464,1.972,5.80,7.076,6.728,3.596,4.272667
2017,0.468,1.989,5.85,7.137,6.786,3.627,4.309500


In [17]:
# GIA in mm/year
(gia_df.loc[2019] - gia_df.loc[2018])*10
# Where does the 1.1 come from in Vermersen?

20      0.040000
22      0.170000
23      0.500000
24      0.610000
25      0.580000
32      0.310000
mean    0.368333
dtype: float64

In [19]:
lev =  slc.LevitusSL(extrap=True, extrap_back=True)
lev

,StericLevitus
time,
1950,-1.480427
1951,-1.442732
1952,-1.405037
1953,-1.367342
1954,-1.329647
...,...
2016,2.662499
2017,2.710927
2018,2.784583


In [31]:
a = 7
if a:
    print('Executing')

Executing
